# Figure 1


In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.patches as mpatches


In [2]:
pop_df=pd.read_csv(r'C:\Users\Yasaman\Downloads\World_bank_population.csv',skiprows=3)
pop_df=pop_df[['Country Code','2019']].dropna()
pop_df['2019']=pop_df['2019'].astype(int)
possible_countries=pop_df.query(" `2019` >=1000000")['Country Code'].values
possible_countries=[x.lower() for x in possible_countries]

In [3]:
excluded_iso3_codes = [
    "IRL",  # Ireland
    "SSD",  # South Sudan
    "SDN",  # Sudan
    "COG",  # Republic of the Congo
    "COD",  # Democratic Republic of the Congo
    "GIN",  # Guinea
    "GNB",  # Guinea-Bissau
    "GNQ",  # Equatorial Guinea
    "PNG",  # Papua New Guinea
    "XKX",  # Kosovo (unofficial)
    "MNE",  # Montenegro
    "SRB",  # Serbia
    "TLS"   # Timor-Leste
]
excluded_iso3_codes=[c.lower() for c in excluded_iso3_codes]

In [4]:
possible_iso=list(set(possible_countries)-set(excluded_iso3_codes))

In [61]:

df=pd.read_csv(r"C:\Users\Yasaman\Downloads\Attention-fractional counting.csv")
df=df[df['country'].isin(possible_iso)]
df=df.rename(columns={'year':'Year', 'aggregated_value':'count', 'country':'Mention_country', 'affiliation_country':'Aff_country'})
Country_list={'Egypt':'EGY', 'Tunisia':'TUN','Libya':'LBY','Syria':'SYR','Yemen':'YEM','Bahrain':'BHR','Jordan':'JOR','Kuwait':'KWT','Morocco':'MAR','Oman':'OMN'}
rev_Country_list={Country_list[key]: key for key in Country_list}
abbr=[country.lower() for country in Country_list.values()]
physical_sciences=[ 'MATH', 'ENGI', 'PHYS', 'COMP']
df=df[~df['subjarea'].isin(physical_sciences)]
df=df.groupby(['Year', 'Mention_country', 'Aff_country'])['count'].sum().reset_index()
    

In [62]:

df=df[df['Mention_country'].str.lower().isin(abbr)]

In [63]:

country_codes=pd.read_csv(r"C:\Users\Yasaman\Downloads\iso3.csv")
country_codes['iso3']=[c.lower() for c in country_codes['iso3']]
Map={country_codes.iloc[c]['iso3']: country_codes.iloc[c]['name'] for c in range(len(country_codes))}
Map['irn']='Iran'
Map['usa']='USA'
Map['gbr']='UK'
Map['syr']='Syria'

# Define year ranges for "before" and "after" data.
years_before = np.arange(2002, 2011, 1)
years_after = np.arange(2011, 2020, 1)



# Function to calculate summary statistics for given years and data.
def summarize_years(df, years):
    # Filter the DataFrame for the given years range.
    df_filtered = df[(df['Year'].isin(years))]
    # Sum of counts by affiliation country.
    summary_df = df_filtered.groupby(['Aff_country','Mention_country'])['count'].sum().reset_index()

    return summary_df.fillna(0)  # Fill NaN with 0 if any.

# Apply the function to both year ranges.
before_df = summarize_years(df, years_before)
before_df = before_df.add_suffix('_before')

after_df = summarize_years(df, years_after)
after_df = after_df.add_suffix('_after')


full_df = before_df.merge(after_df, left_on=['Aff_country_before','Mention_country_before'], right_on=['Aff_country_after','Mention_country_after'], how='outer')
full_df.fillna(0, inplace=True)
full_df['Mention_country'] = np.where(full_df['Mention_country_before'] != 0, full_df['Mention_country_before'], full_df['Mention_country_after'])
full_df['Aff_country'] = np.where(full_df['Aff_country_before'] != 0, full_df['Aff_country_before'], full_df['Aff_country_after'])
full_df.drop(columns=['Aff_country_before','Mention_country_before','Mention_country_after','Aff_country_after'], inplace=True)


In [64]:
paper_df=full_df.groupby('Mention_country').sum().reset_index().drop(columns=['Aff_country'])

In [65]:
paper_df['Tot']=paper_df['count_before']+paper_df['count_after']

In [66]:
paper_df['per_before']=100*paper_df['count_before']/paper_df['count_before'].sum()
paper_df['per_after']=100*paper_df['count_after']/paper_df['count_after'].sum()

In [67]:
paper_df['per_tot']=100*paper_df['Tot']/paper_df['Tot'].sum()   

In [68]:
paper_df['change_per']=paper_df['per_after']-paper_df['per_before']
paper_df['change_number']=paper_df['count_after']-paper_df['count_before']

In [69]:
paper_df

,Mention_country,count_before,count_after,Tot,per_before,per_after,per_tot,change_per,change_number
0,bhr,393.567235,814.669726,1208.236961,2.023522,1.768037,1.843870,-0.255484,421.102491
1,egy,5241.468858,15170.778355,20412.247213,26.948956,32.924389,31.150778,5.975433,9929.309497
2,jor,3143.133347,6606.324628,9749.457975,16.160387,14.337379,14.878479,-1.823007,3463.191281
3,kwt,1535.301025,2086.928709,3622.229734,7.893734,4.529158,5.527822,-3.364576,551.627685
4,lby,491.956415,1388.326988,1880.283403,2.529389,3.013017,2.869468,0.483629,896.370572
5,mar,2876.955401,6550.969596,9427.924997,14.791835,14.217245,14.387793,-0.574590,3674.014195
6,omn,1420.167178,3071.648539,4491.815717,7.301774,6.666247,6.854882,-0.635528,1651.481361
7,syr,753.932160,2380.623439,3134.555599,3.876334,5.166549,4.783591,1.290215,1626.691279
8,tun,3014.819813,6706.885910,9721.705723,15.500664,14.555623,14.836127,-0.945041,3692.066098
9,yem,578.315983,1300.474162,1878.790145,2.973405,2.822355,2.867189,-0.151051,722.158179


In [ ]:

result_df=full_df.groupby('Mention_country')[['count_after', 'count_before']].sum().reset_index()

result_df['count_after']/=len(years_before)
result_df['count_before']/=len(years_before)
result_df['Absolute difference']=result_df['count_after']-result_df['count_before']

col='Absolute difference'
markers=['o', 'v', '^', 'x', 's', 'd', '>', '<', 'p', '+']
copy_df = result_df[result_df['Mention_country'].isin(abbr)]
copy_df=copy_df.sort_values(by=col, ascending=True).reset_index(drop=True)

country_names=['egy','tun', 'lby', 'syr', 'yem', 'bhr', 'jor', 'kwt', 'mar', 'omn']

In [6]:
result_df

,Mention_country,count_after,count_before,Absolute difference
0,afg,346.630504,115.536695,231.093809
1,ago,125.928746,55.175790,70.752956
2,alb,152.440302,52.523437,99.916865
3,are,536.508385,207.082800,329.425586
4,arg,2258.258568,1205.171588,1053.086980
...,...,...,...,...
143,vnm,1331.012960,477.721076,853.291884
144,yem,144.497129,64.257331,80.239798
145,zaf,4073.589392,1760.643760,2312.945632
146,zmb,321.030011,121.005866,200.024144


In [62]:
result_df['percentile']=result_df['Absolute difference'].rank(pct=True)
copy_df = result_df[result_df['Mention_country'].isin(abbr)]


In [63]:
copy_df

,Mention_country,count_after,count_before,Absolute difference,percentile
15,bhr,90.518858,43.729693,46.789166,0.148649
39,egy,1685.642039,582.385429,1103.256611,0.810811
65,jor,734.036070,349.237039,384.799031,0.581081
72,kwt,231.880968,170.589003,61.291965,0.202703
76,lby,154.258554,54.661824,99.596730,0.283784
81,mar,727.885511,319.661711,408.223799,0.594595
102,omn,341.294282,157.796353,183.497929,0.445946
127,syr,264.513715,83.770240,180.743475,0.439189
134,tun,745.209546,334.979979,410.229566,0.601351
144,yem,144.497129,64.257331,80.239798,0.236486


In [66]:
result_df.sort_values('percentile', ascending=True)[20:30]

,Mention_country,count_after,count_before,Absolute difference,percentile
141,uzb,98.079901,51.355659,46.724242,0.141892
15,bhr,90.518858,43.729693,46.789166,0.148649
36,dom,107.162135,57.283776,49.878358,0.155405
97,nic,133.520587,82.721112,50.799474,0.162162
109,prk,79.358443,27.944148,51.414295,0.168919
95,ner,86.314246,33.591342,52.722904,0.175676
131,tjk,74.065878,21.092367,52.973510,0.182432
82,mda,77.528582,23.265745,54.262838,0.189189
17,blr,112.952868,57.090000,55.862868,0.195946
72,kwt,231.880968,170.589003,61.291965,0.202703
